In [28]:
from __future__ import print_function

from argparse import Namespace

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions

In [29]:
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)  # n_units -> n_out

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

In [30]:
def main(args):
    model = L.Classifier(MLP(args.unit, 10))
    if args.gpu >= 0:
        # Make a specified GPU current
        chainer.cuda.get_device_from_id(args.gpu).use()
        model.to_gpu()  # Copy the model to the GPU

    optimizer = chainer.optimizers.Adam()
    optimizer.setup(model)

    train, test = chainer.datasets.get_mnist()

    train_iter = chainer.iterators.SerialIterator(train, args.batchsize)
    test_iter = chainer.iterators.SerialIterator(test, args.batchsize,
                                                 repeat=False, shuffle=False)

    updater = training.StandardUpdater(train_iter, optimizer, device=args.gpu)
    trainer = training.Trainer(updater, (args.epoch, 'epoch'), out=args.out)

    trainer.extend(extensions.Evaluator(test_iter, model, device=args.gpu))

    # Take a snapshot for each epoch
    trainer.extend(extensions.snapshot(), trigger=(1, 'epoch'))

    trainer.extend(extensions.LogReport())

    trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss',
         'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))
    
    if args.resume:
        # Resume from a snapshot
        chainer.serializers.load_npz(args.resume, trainer)
        
    if args.freeze:
        # freeze all layers but the last fc layer
        model.predictor.disable_update()
        model.predictor.l3.enable_update()
        
    print('freezing previous layers: ',
          not model.predictor.l1.update_enabled)
    
    # Run the training
    trainer.run()

In [31]:
args = {
        'batchsize': 100,
        'epoch': 5,
        'gpu': 0,
        'out': 'result',
        'resume': False,
        'unit': 1000,
        'freeze': False,
       }
args = Namespace(**args)

## Sometimes we want to first freezing the earlier layers

In [34]:
args.freeze = True
args.resume = None
main(args)

freezing previous layers:  True
epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           0.824268    0.440981              0.8353         0.8977                    1.98654       
2           0.395162    0.327922              0.901201       0.9174                    4.42113       
3           0.318553    0.282001              0.916518       0.9246                    6.76895       
4           0.279653    0.255968              0.924701       0.9302                    9.10749       
5           0.253995    0.237168              0.930568       0.934                     11.4499       


## When we want to resume training for all layers, from a previous snapshot that freezes layers, such error occurs

In [35]:
args.freeze = False
args.resume = 'result/snapshot_iter_3000'
main(args)

KeyError: 'updater/optimizer:main/predictor/l2/W/m is not a file in the archive'